In [1]:
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import ultron.factor.empyrical as empyrical

from lumina.genetic.metrics.ts_pnl import calculate_ful_ts_ret
from lumina.genetic.signal.method import *
from lumina.genetic.strategy.method import *

from kdutils.macro2 import *
from kdutils.tactix import Tactix


In [3]:
def read_factors(method, instruments, task_id, period, name):
    dirs = os.path.join(base_path, method, instruments, 'temp', "model",
                        str(task_id), str(period))
    filename = os.path.join(dirs, "{0}_predict_data.feather".format(name))
    predict_data = pd.read_feather(filename)
    is_on_mark = predict_data['trade_time'].dt.minute % int(period) == 0
    predict_data = predict_data[is_on_mark]
    predict_data.replace([np.inf, -np.inf], np.nan, inplace=True)
    predict_data.dropna(inplace=True)
    return predict_data

In [4]:
def signal_function(factors_data, signal_method, signal_params):
    pos_data = eval(signal_method)(factor_data=factors_data, **signal_params)
    return pos_data


def strategy_function(pos_data, total_data, strategy_method, strategy_params):
    pos_data1 = eval(strategy_method)(signal=pos_data,
                                      total_data=total_data,
                                      **strategy_params)
    return pos_data1

In [5]:
method = 'cicso0'
instruments = 'ims'
task_id = '200037'
period = 5
name = 'linear'
signal_method = 'adaptive_signal'
signal_params = {'roll_num': 25, 'threshold': 0.9}
strategy_settings = {'commission': 0.000023, 'slippage': 0, 'size': 200}

In [6]:
predict_data = read_factors(method=method,
                                instruments=instruments,
                                task_id=task_id,
                                period=period,
                                name=name)
predict_data.head()

,trade_time,code,predict,nxt1_ret_5h
0,2024-12-05 10:20:00,IM,-5.284011,-0.001860
5,2024-12-05 10:25:00,IM,-0.186690,-0.000206
10,2024-12-05 10:30:00,IM,-1.949980,0.003050
15,2024-12-05 10:35:00,IM,-0.980163,0.004669
20,2024-12-05 10:40:00,IM,-5.241567,0.000327


In [7]:
total_data2 = predict_data.set_index(['trade_time', 'code']).unstack()
total_data2.head()

,predict,nxt1_ret_5h
code,IM,IM
trade_time,,
2024-12-05 10:20:00,-5.284011,-0.001860
2024-12-05 10:25:00,-0.186690,-0.000206
2024-12-05 10:30:00,-1.949980,0.003050
2024-12-05 10:35:00,-0.980163,0.004669
2024-12-05 10:40:00,-5.241567,0.000327


In [8]:
factors_data = predict_data.set_index(
        ['trade_time',
         'code'])[['predict']].rename(columns={'predict': 'transformed'})
factors_data.head()

,,transformed
trade_time,code,
2024-12-05 10:20:00,IM,-5.284011
2024-12-05 10:25:00,IM,-0.186690
2024-12-05 10:30:00,IM,-1.949980
2024-12-05 10:35:00,IM,-0.980163
2024-12-05 10:40:00,IM,-5.241567


In [9]:
pos_data = eval(signal_method)(factor_data=factors_data, **signal_params)

In [10]:
df = calculate_ful_ts_ret(pos_data=pos_data,
                              total_data=total_data2,
                              strategy_settings=strategy_settings,
                              ret_name='nxt1_ret_5h')

In [15]:
df['a_ret'].head()

trade_time
2024-12-05    0.000000
2024-12-06   -0.011649
2024-12-07    0.000000
2024-12-08    0.000000
2024-12-09   -0.006767
Freq: D, Name: a_ret, dtype: float64

In [14]:
pos_data.head()

,pos
,IM
trade_time,
2024-12-05 10:20:00,0
2024-12-05 10:25:00,0
2024-12-05 10:30:00,0
2024-12-05 10:35:00,0
2024-12-05 10:40:00,0
